In [1]:
from keras.callbacks import ModelCheckpoint

import numpy
import math
import cv2
import time

from utility.h5data import h5DataRead

Using TensorFlow backend.


In [2]:
EPOCHS = 10
BATCH_SIZE = 128

SCALE = 4
FILENAME_TRAIN = "dataTrain_" + str(SCALE) + ".h5"
FILENAME_VALIDATE = "dataValidate_" + str(SCALE) + ".h5"
FILENAME_CHECKPOINT = "dataCheckPoint_" + str(SCALE) + ".h5"

In [3]:
# dnn configuration for waifu2x
from network.waifu2x import waifu2x
dnn = waifu2x()
dnn.in_train = True

In [4]:
tick1 = cv2.getTickCount()

In [ ]:
model = dnn.network()
print(model.summary())
data, label = h5DataRead(FILENAME_TRAIN)
val_data, val_label = h5DataRead(FILENAME_VALIDATE)

checkpoint = ModelCheckpoint(FILENAME_CHECKPOINT, monitor='val_loss', verbose=1, save_best_only=True,
                             save_weights_only=False, mode='min')

callbacks_list = [checkpoint]

model.fit(data, label, batch_size=BATCH_SIZE, validation_data=(val_data, val_label),
                callbacks=callbacks_list, shuffle=True, epochs=EPOCHS, verbose=1)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 32)        9248      
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
leaky_re_lu_3 (LeakyReLU)    (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 32, 32, 64)        36928     
__________

In [ ]:
score = model.evaluate(val_data, val_label, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:
fileName = 'yayoi_' + str(SCALE) + "_" + time.strftime("%Y-%m-%d_%H%M%S", time.localtime()) + '.h5'
model.save_weights(fileName)
print(fileName + ' saved')

In [ ]:
tick2 = cv2.getTickCount()
tick = math.floor( ((tick2 - tick1) * 1000) / cv2.getTickFrequency())

if tick >= 60000:
    mins = math.floor(tick / 60000)
    secs = math.floor((tick - mins * 60000) / 1000)
    msec = tick - mins * 60000 - secs * 1000
    print("processed time: " + str(mins) + " mins " + str(secs) + " secs " + str(msec) + " ms")
elif tick >= 1000:
    secs = math.floor(tick / 1000)
    msec = tick - secs * 1000
    print("processed time: " + str(secs) + " secs " + str(msec) + " ms")
else:
    print("processed time: " + str(tick) + " ms")